In [5]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langserve import RemoteRunnable
from langchain_core.runnables import RunnableParallel
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from operator import itemgetter

In [2]:
from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [3]:
from langchain_community.chat_models import ChatOllama

# llm = ChatOllama(model="llama3:latest")
# llm = ChatOllama(model="Llama-3-Open-Ko-8B-FP16.gguf:latest")
llm = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest")

In [7]:
llm.invoke("What is stock?")

AIMessage(content='Stock refers to two main concepts in different contexts:\n\n1. Financial markets: In the financial market context, "stock" or "equity" refers to shares of ownership in a company that are traded on public exchanges like the New York Stock Exchange (NYSE) or NASDAQ. When you buy stocks, you essentially become a part-owner of the company and have a claim to its assets and earnings. The value of your investment can fluctuate based on market conditions, corporate performance, and other factors.\n\n2. Inventory: In business and supply chain management, "stock" refers to the items that a company holds in inventory for sale or distribution. This includes raw materials, work-in-progress goods, and finished products. The goal of managing stock is to maintain an optimal balance between having enough inventory to meet customer demand while minimizing storage costs and avoiding overstocking.\n\nIn summary, "stock" can refer to either shares in a company or the items a business ho

# PromptTemplate

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 권봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what. "),
    ("user", "{input}")
])

chain = prompt | llm
chain.invoke({"input": "What is stock?"})

# AIMessage(content='안녕하세요! 저는 도움이 되고 전문적인 조수인 권봇입니다. 궁금한 점이나 요청사항이 있으시면 언제든지 물어보세요. 최선을 다해 도와드리겠습니다.\n\n주식(Stock)이란 무엇인가요?\n\n주식은 한 회사의 소유권을 나타내는 증권으로, 투자자들이 그 회사의 일부를 소유하게 됩니다. 주식을 구매하면 해당 회사의 수익과 손실에 대한 비례적인 지분을 가지게 되며, 배당금이나 미래 주식 가치 상승을 통해 이익을 얻을 수 있습니다. 또한, 주식을 매매함으로써 자본 이득을 실현할 수도 있습니다.\n\n주식은 다양한 산업과 규모의 기업에서 발행되며, 투자자들은 자신의 목표와 위험 감수 성향에 따라 다양한 유형의 주식을 선택할 수 있습니다. 예를 들어, 성장주를 선호하는 투자자라면 빠르게 성장하는 회사에 투자하는 것을 고려할 수 있고, 안정성을 중시하는 투자자라면 안정적인 배당금을 제공하는 회사나 블루칩을 선택할 수도 있습니다.\n\n주식 투자는 자본 이득과 배당금 지급으로 인한 소득을 제공할 수 있지만, 시장 변동성, 기업 성과에 대한 불확실성, 그리고 특정 산업이나 회사의 위험 등 다양한 위험이 따릅니다. 따라서 투자 결정을 내리기 전에 철저한 조사를 하고 재무 고문과 상담하는 것이 중요합니다.', response_metadata={'model': 'EEVE-Korean-Instruct-10.8B-v1.0:latest', 'created_at': '2024-07-01T11:05:48.894585Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 34985160791, 'load_duration': 1134154833, 'prompt_eval_count': 49, 'prompt_eval_duration': 1024227000, 'eval_count': 261, 'eval_duration': 32818249000}, id='run-0e85851b-44ac-4555-87c6-f6da4bb9ac37-0')
# SystemMessage(content='You are a helpful, professional assistant named 권봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what. ')
# HumanMessage(content='{input}')

AIMessage(content='안녕하세요! 저는 도움이 되고 전문적인 조수인 권봇입니다. 주식에 대해 설명해 드리겠습니다.\n\n주식은 한 회사의 소유권을 나타내는 증권으로, 투자자들이 그 회사의 일부를 소유하게 됩니다. 즉, 주식을 구매하면 해당 회사의 지분을 취득하는 것입니다. 이러한 방식으로 개인이나 기업이 다른 기업에 투자할 수 있습니다.\n\n예를 들어, 당신이 A라는 회사에 100만 원을 투자하여 주식 100주를 구입했다고 가정해 보겠습니다. 그 후 A회사의 주가가 상승한다면, 당신은 주식을 매도하여 이익을 실현할 수 있습니다. 반대로, 회사의 가치가 하락하면 주식이 가치가 줄어들어 손실을 볼 수도 있습니다.\n\n주식은 다양한 이유로 투자자들이 구매합니다:\n\n1. 자본 가치 상승: 주식의 가격이 시간이 지남에 따라 증가함에 따라 투자자는 수익을 얻을 수 있습니다.\n2. 배당금 지급: 일부 회사는 주식을 보유한 사람들에게 정기적으로 현금 분배(배당금)를 제공합니다. 이는 회사의 이익 일부를 공유하는 방법입니다.\n3. 다양화: 다양한 산업에 걸쳐 여러 회사에 투자하는 것은 위험을 줄이고 포트폴리오의 안정성을 높일 수 있습니다.\n4. 성장 잠재력: 일부 회사는 미래 수익성이 높은 것으로 예상되어 투자자들에게 매력적인 투자가 될 수 있습니다.\n5. 인수합병(M&A): 때때로 한 회사가 다른 회사를 인수하여 주식을 보유한 사람들에게 상당한 이익을 가져다 줄 수도 있습니다.\n\n주식 투자는 위험과 보상을 모두 포함하고 있으므로, 정보에 입각한 결정을 내리기 위해서는 철저한 연구와 이해가 필요합니다. 또한, 주식 투자에 앞서 항상 금융 전문가나 재무 고문과 상담하는 것이 좋습니다.', response_metadata={'model': 'EEVE-Korean-Instruct-10.8B-v1.0:latest', 'created_at': '2024-08-22T12:43:31.079467Z', 'message': {'role': 'assistant',

# StrOutputParser

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | llm | StrOutputParser()
chain.invoke({"input": "What is stock?"})

'안녕하세요! 저는 도움이 되고 전문적인 조수인 권봇입니다. 궁금한 점을 도와드리겠습니다. 주식에 대해 설명해 드리겠습니다.\n\n주식은 한 회사의 소유권을 나타내는 증권으로, 투자자들이 그 회사의 일부를 소유하게 됩니다. 예를 들어, 어떤 회사가 100개의 주식을 발행했다면, 각 주식을 구매한 사람은 해당 회사의 1%를 소유하게 됩니다. 이러한 방식으로 사람들은 다양한 사업에 투자하고 수익을 창출할 수 있습니다.\n\n주식은 거래소에서 사고팔 수 있으며, 주식의 가격은 공급과 수요에 따라 변동됩니다. 또한, 기업들은 보통 매년 배당금을 지급하는데, 이는 투자자들에게 주식을 보유하고 있는 대가로 받는 수익입니다.\n\n요약하자면, 주식은 사람들이 회사의 일부를 소유하고 수익을 창출할 수 있게 해주는 금융 상품으로, 거래소에서 사고팔 수 있으며 배당금 형태로 수익을 얻을 수 있습니다.'

# StructuredOutputParser

In [4]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(
        name="answer", 
        description="사용자의 질문에 대한 답변이다."),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답변하는 데 사용된 출처나 웹사이트여야 한다."),
]
structured_output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = structured_output_parser.get_format_instructions()
format_instructions
llm.with_structured_output

<bound method BaseChatModel.with_structured_output of ChatOllama(model='EEVE-Korean-Instruct-10.8B-v1.0:latest')>

In [10]:
structured_output_prompt = ChatPromptTemplate.from_messages([
    ("system", "사용자의 질문에 최선을 다해 답변해야 한다. 한국어로 대답해야 한다.\n{format_instructions}\n"),
    ("user", "{question}")
])

structured_chain = (
    {"format_instructions": RunnableLambda(lambda _: format_instructions), "question": RunnablePassthrough()}
    | structured_output_prompt 
    | llm 
    | structured_output_parser
)
structured_chain.invoke({"question": "주식이란?"})

{'answer': '주식은 한 회사의 소유권을 나타내는 증권입니다. 이는 투자자들이 그 회사의 일부를 소유하고 있으며, 회사의 이익이나 손실에 대한 권리를 가짐을 나타냅니다. 주식을 구매함으로써 개인은 해당 회사의 성과에 기여하게 되며, 배당금과 잠재적 미래 가치 상승의 혜택을 받을 수 있습니다.',
 'source': 'https://www.investopedia.com/terms/s/stock.asp'}

In [25]:
extract_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
당신은 텍스트에서 여러 주식 관련 정보를 추출하는 전문가입니다. 주어진 텍스트를 분석하고 언급된 모든 주식에 대해 다음 정보를 추출하여 구조화된 형식으로 제공해 주세요:

- 기업명 (company): 주식을 발행한 회사 이름
- 종목코드 (ticker): 주식의 종목 코드 (언급된 경우)
- 산업 (industry): 기업이 속한 산업 분야
- 현재가 (current_price): 주식의 현재 가격 (숫자로 표시, 원 단위)
- 목표가 (target_price): 애널리스트의 목표 가격 (숫자로 표시, 원 단위)
- 시가총액 (market_cap): 기업의 시가총액 (숫자로 표시, 원 단위)
- 주요 지표 (key_metrics): PER, PBR 등 주요 재무 지표 (딕셔너리 형태)
- 주요 뉴스 (recent_news): 최근 주요 뉴스나 이벤트 (리스트 형태)

위의 정보 중 텍스트에서 명확하게 언급되지 않은 항목은 null로 표시하세요.

추출한 정보를 다음과 같은 JSON 형식의 리스트로 제공해 주세요:

```json
[
  {{
    "company": "기업명",
    "ticker": "종목코드",
    "industry": "산업",
    "current_price": 현재가,
    "target_price": 목표가,
    "market_cap": 시가총액,
    "key_metrics": {{
      "PER": PER 값,
      "PBR": PBR 값
    }},
    "recent_news": ["뉴스1", "뉴스2", ...]
  }},
  {{
    // 다음 주식 정보
  }}
]
```

예시:
입력: "삼성전자(005930)의 주가가 7만원을 돌파했습니다. 현재 주가는 72,000원이며, 애널리스트들의 평균 목표가는 85,000원입니다. 시가총액은 430조원으로 국내 1위를 유지하고 있습니다. 현재 PER은 13.5, PBR은 1.2입니다. 최근 AI 반도체 개발 성공 소식이 알려져 주가 상승을 이끌었습니다."

출력:
```json
[
  {{
    "company": "삼성전자",
    "ticker": "005930",
    "industry": "전자",
    "current_price": 72000,
    "target_price": 85000,
    "market_cap": 430000000000000,
    "key_metrics": {{
      "PER": 13.5,
      "PBR": 1.2
    }},
    "recent_news": ["AI 반도체 개발 성공"]
  }}
]
```

주어진 텍스트를 분석하고 위의 형식에 맞춰 모든 언급된 주식 정보를 추출해 주세요. 텍스트에 여러 주식이 언급된 경우, 각 주식에 대한 정보를 별도의 객체로 리스트에 포함시켜 주세요.
            """,
        ),
        ("human", "{text}"),
    ]
)

extract_chain = extract_prompt | llm | StrOutputParser()
text = """
최근 금융투자업계에 따르면, 인공지능(AI) 관련 기업들의 주가가 급등하고 있습니다. 특히 네이버(035420)와 카카오(035720)의 주가 상승이 두드러집니다. 네이버의 현재 주가는 216,000원으로 전일 대비 5% 상승했으며, 애널리스트들의 평균 목표가는 250,000원입니다. 네이버의 시가총액은 35조원을 돌파했으며, 현재 PER은 18.5, PBR은 2.3입니다. 최근 하이퍼클로바X 출시 소식이 알려져 주가 상승을 견인했습니다.
카카오의 경우, 현재 주가는 52,000원으로 전일 대비 3% 상승했습니다. 애널리스트들의 평균 목표가는 65,000원이며, 시가총액은 23조원을 기록했습니다. 카카오의 PER은 20.1, PBR은 1.8입니다. 카카오는 최근 AI 기반 금융 서비스 확대 계획을 발표해 투자자들의 관심을 끌고 있습니다.
한편, 반도체 업종에서는 삼성전자(005930)와 SK하이닉스(000660)의 주가가 주목받고 있습니다. 삼성전자의 현재 주가는 72,800원으로 소폭 상승했으며, 목표가는 85,000원입니다. 시가총액은 435조원으로 국내 1위를 유지하고 있습니다. SK하이닉스는 현재 133,000원에 거래되고 있으며, 애널리스트들은 160,000원까지 상승할 것으로 전망하고 있습니다. 두 기업 모두 AI 반도체 시장 선점을 위한 투자 확대 소식이 전해지면서 주가에 긍정적인 영향을 미치고 있습니다.
제약바이오 섹터에서는 셀트리온(068270)이 주목받고 있습니다. 셀트리온의 주가는 현재 178,000원으로, 최근 유럽에서의 바이오시밀러 점유율 확대 소식에 힘입어 강세를 보이고 있습니다. 애널리스트들의 목표가는 평균 220,000원이며, 시가총액은 25조원을 넘어섰습니다.
투자자들은 이러한 주요 기업들의 실적 발표와 함께, 글로벌 경제 상황 및 미국 연준의 금리 정책 등을 주시하며 투자 전략을 수립하고 있습니다.
"""
extract_chain.invoke({"text": text})

'```json\n[\n  {\n    "company": "네이버",\n    "ticker": "035420",\n    "industry": "전자",\n    "current_price": 216000,\n    "target_price": 250000,\n    "market_cap": 35000000000000,\n    "key_metrics": {\n      "PER": 18.5,\n      "PBV": 2.3\n    },\n    "recent_news": ["하이퍼클로바X 출시"]\n  },\n  {\n    "company": "카카오",\n    "ticker": "035720",\n    "industry": "전자",\n    "current_price": 52000,\n    "target_price": 65000,\n    "market_cap": 23000000000000,\n    "key_metrics": {\n      "PER": 20.1,\n      "PBV": 1.8\n    },\n    "recent_news": ["AI 기반 금융 서비스 확대 계획 발표"]\n  },\n  {\n    "company": "삼성전자",\n    "ticker": "005930",\n    "industry": "반도체",\n    "current_price": 72800,\n    "target_price": 85000,\n    "market_cap": 43500000000000000,\n    "key_metrics": {\n      "PER": null,\n      "PBV": null\n    },\n    "recent_news": ["AI 반도체 시장 선점을 위한 투자 확대"]\n  },\n  {\n    "company": "SK하이닉스",\n    "ticker": "000660",\n    "industry": "반도체",\n    "current_price": 133000,\n    "target_pri

# 체인 비교

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 권봇. answer me in Korean no matter what"),
    ("user", "{input}")
])

# chain = prompt | llm | output_parser
my_input = {"input": "What is stock?"}
print(f'1st chain >> {prompt.invoke(my_input)}')
print(f'2nd chain >> {(prompt | llm).invoke(my_input)}')
print(f'3rd chain >> {(prompt | llm | StrOutputParser()).invoke(my_input)}')

1st chain >> messages=[SystemMessage(content='You are a helpful, professional assistant named 권봇. answer me in Korean no matter what'), HumanMessage(content='What is stock?')]
2nd chain >> content='주식은 한 기업에서 소유한 지분 또는 일부를 의미합니다. 투자자들이 회사의 성장과 성공에 기여하고자 자본을 제공함으로써 취득하게 됩니다. 주식을 구매하면 해당 기업의 일부가 되며, 그 성과에 따라 배당금이나 미래 주식 가치 상승으로 인한 이익을 얻게 됩니다. 또한, 주식을 매매하여 가격 변동에 따른 차익을 실현할 수도 있습니다.' response_metadata={'model': 'EEVE-Korean-Instruct-10.8B-v1.0:latest', 'created_at': '2024-05-06T03:09:00.611458Z', 'message': {'role': 'assistant', 'content': ''}, 'done': True, 'total_duration': 16761211959, 'load_duration': 6395765875, 'prompt_eval_count': 40, 'prompt_eval_duration': 979833000, 'eval_count': 79, 'eval_duration': 9379067000} id='run-23955113-0cc5-4ad0-a48a-7accd66da0de-0'
3rd chain >> 주식은 한 기업에서 소유한 지분 또는 일부를 의미합니다. 투자자가 회사의 일부분을 구매함으로써 해당 기업의 성과에 대한 재정적 이익을 얻을 수 있습니다. 주식은 공개 시장에서 거래되며, 이는 사람들이 증권 거래소나 온라인 플랫폼을 통해 주식을 사고팔 수 있음을 의미합니다. 이러한 방식으로 개인들은 다양한 산업에 걸쳐 있는 기업들에 투자할 기회를 가질 수 있으며, 이는 결국 

# Stream

In [12]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 권봇. Introduce yourself first, and answer the questions. answer me in Korean no matter what."),
    ("user", "{input}")
])

chain = prompt | llm | StrOutputParser()
for token in chain.stream(
    {"input": "What is stock?"}
):
    print(token, end="")

주식은 한 기업에 대한 소유권 또는 지분을 의미합니다. 투자자가 회사의 일부를 구매함으로써 해당 기업의 성과에 참여할 수 있게 됩니다. 주식을 매입하면, 그 투자자는 배당금 지급이나 주식 가치 상승으로 인한 잠재적 이득을 기대할 수 있습니다. 또한, 주주는 회사 의사 결정 과정에 참여하고, 주주총회에서 투표하며, 회사의 미래 방향성에 영향을 미칠 수 있는 권리를 가집니다.

# Combined chain

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

# 첫번째 체인
prompt1 = ChatPromptTemplate.from_template("[{korean_input}] translate the question into English. Don't say anything else, just translate it.")
chain1 = (
    prompt1 
    | llm 
    | StrOutputParser()
)
message1 = chain1.invoke({"korean_input": "주식이 뭐야?"})
print(f'message1: {message1}')

# 두번째 체인 (첫번째 체인 연결)
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful, professional assistant named 권봇. answer me in Korean no matter what"),
    ("user", "{input}")
])
chain2 = (
    {"input": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)
message2 = chain2.invoke({"korean_input": "주식이 뭐야?"})
print(f'message2: {message2}')

message1: What is a stock?
message2: 주식은 한 회사의 소유권을 나타내는 증권입니다. 주식을 구매하면 해당 회사의 일부를 소유하게 되며, 이는 회사 성과에 따른 배당금이나 미래 주식 가치 상승으로 인한 이익을 얻을 수 있는 권리를 가지게 됩니다. 간단히 말해서, 주식을 사는 것은 그 회사의 일부분을 사는 것과 같습니다.


# RunnableLambda

In [55]:
def length_function(text):
    print(f"length_function / text: {text}")
    return len(text)

def _multiple_length_function(text1, text2):
    print(f"_multiple_length_function / text1: {text1} / text2: {text2}")
    return len(text1) * len(text2)

def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

prompt = ChatPromptTemplate.from_template("what is {a} + {b}? Don't say anything else, just give the right answer.")

chain1 = prompt | llm

# chain = (
#     RunnableLambda(lambda x: {
#         "a": length_function(x["foo"])
#         , "b": multiple_length_function({"text1": x["foo"], "text2": x["bar"]})
#         })
#     | prompt
#     | llm
# )
# 위 체인과 동일
chain = (
    {
        "a": itemgetter("foo") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("foo"), "text2": itemgetter("bar")} | RunnableLambda(multiple_length_function),
    }
    | prompt
    | llm
)

In [56]:
answer = chain.invoke({"foo": "aa", "bar": "bbb"})
print(answer.content)

length_function / text: aa
_multiple_length_function / text1: aa / text2: bbb
The correct answer is 8.


# RunnableParallel

In [14]:
joke_chain = (
    ChatPromptTemplate.from_template("{topic}에 관련해서 짧은 농담 말해줘") 
    | llm)
poem_chain = (
    ChatPromptTemplate.from_template("{topic}에 관련해서 시 2줄 써줘") 
    | llm)

# map_chain = {"joke": joke_chain, "poem": poem_chain} # 체인에서 이처럼 사용할 때, 자동으로 RunnableParallel 사용됨
# map_chain = RunnableParallel({"joke": joke_chain, "poem": poem_chain})
map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "애플"})

{'joke': AIMessage(content="알고 보니 애플이 '아이스버그'를 사들였대요 - 이제 모든 아이폰에는 빙산 앱이 기본으로 설치돼 있답니다!", response_metadata={'model': 'EEVE-Korean-Instruct-10.8B-v1.0:latest', 'created_at': '2024-06-07T06:11:49.386905Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1747483625, 'load_duration': 19981500, 'prompt_eval_count': 16, 'prompt_eval_duration': 379381000, 'eval_count': 36, 'eval_duration': 1345341000}, id='run-80a6ba77-c5ca-4741-a020-909537deea41-0'),
 'poem': AIMessage(content='\n아이폰의 매혹적인 화면이 우리를 사로잡네,\n맥북은 우리의 창의력을 불러일으키며,\n시리야, 언제나 도와주고, 아이패드는 우리 손끝에서 놀랍게 작동해,\n애플 워치, 멋진 액세서리, 항상 함께하지.\n\n아이팟의 음악이 우리를 춤추게 하고, 애플 TV는 엔터테인먼트를 선사해,\n맥 프로와 맥북 프로가 우리의 작업 흐름을 원활하게 만들어줘,\n애플 생태계, 우리 삶을 풍요롭게 하며, 언제나 새로운 것을 발견할 수 있게 해.\n기술의 정점, 우리가 사랑하는 브랜드, 바로 애플이야!', response_metadata={'model': 'EEVE-Korean-Instruct-10.8B-v1.0:latest', 'created_at': '2024-06-07T06:11:55.114393Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done

# RemoteRunnable

In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
message = remote_chain.invoke({
    "input": "What is stock?",
})
print(f'message : {message}')

message : 주식은 한 기업에서 소유한 지분 또는 일부를 의미합니다. 투자자가 회사의 일부분을 구매함으로써 해당 기업의 성과에 대한 재정적 이익을 얻을 수 있습니다. 주식은 공개 시장에서 거래되며, 이는 사람들이 증권 거래소나 온라인 플랫폼을 통해 주식을 사고팔 수 있음을 의미합니다. 이러한 방식으로 개인들은 다양한 산업에 걸쳐 있는 기업들에 투자할 기회를 가질 수 있으며, 이는 결국 그들의 포트폴리오의 다양화와 위험 분산을 가능하게 합니다.
